In [13]:
from datasets import load_dataset

POSITIVE="positive"
NEUTRAL="neutral"
NEGATIVE="negative"

ID_TO_LABEL = {
    0: NEGATIVE,
    1: NEUTRAL,
    2: POSITIVE,
}

dataset = load_dataset("cardiffnlp/tweet_sentiment_multilingual", "english")
target_dataset = dataset["train"]

# Check out what the data looks like:

positives = [ sample["text"] for sample in target_dataset if ID_TO_LABEL[sample["label"]] == POSITIVE ]
neutral = [ sample["text"] for sample in target_dataset if ID_TO_LABEL[sample["label"]] == NEUTRAL ]
negative = [ sample["text"] for sample in target_dataset if ID_TO_LABEL[sample["label"]] == NEGATIVE ]

negative[189]

'"When I\'m soaring on Sunday afternoon, I learn Frank Gifford--one of my faves on the field and inside the broadcast booth--has died." '